In [1]:
import gspread
from google.oauth2.service_account import Credentials
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import time
import openai

In [ ]:
openai.api_key = ''
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
sheet_path = r"sheets-5e3cf7f4981a.json"
sheet_url = ""

In [3]:
def check_with_llm(html_data):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": f"From the following website's html code, extract these 3 attributes for the website: 'Owner's Name', 'Contact Email', and the 'Phone Number. If you can't find any of these, return for that particular attribute 'Not available'. The returned output should not contain anythign else, it should simply contain the 3 attributes which will be separated by a comma only. Note that the Phone number need not be US-based, it could be any country so therefore expect it to have any country code or differing formats. If the Owner's Name is not explicitly stated, look for nicknames or entity names used in statements like 'Owned by XYZ'. Example Output Format: Mark Henry,support@microsoft.com,32-2-123-4567. Do not return anything else, no note or extra statement at all. Html Code: {html_data}"}]
    )

    output = response.choices[0].message.content.strip()
    result = output.split(',')

    if len(result) == 3:
        data_dict = {
            "Owner's Name": result[0].strip(),
            "Contact Email": result[1].strip(),
            "Phone Number": result[2].strip()}
    else:
        data_dict = {
            "Owner's Name": "Arjeet Banjani",
            "Contact Email": "info@sys.com",
            "Phone Number": "32 2 123 4567"}
        
    return data_dict

In [4]:
check_with_llm("0301-4454717 hfionwgvhpof gwklfhwls <fbdikbfdosl> Owned by Arnie Richards<taggie tag>coochue melon babieee fbdskjbgo@wbogfnrwpobv@odsbv@ojdsblokvndsonfld info@gmail.com <tag>")

{"Owner's Name": 'Arnie Richards',
 'Contact Email': 'info@gmail.com',
 'Phone Number': 'Not available'}

In [ ]:
# Selenium setup (ensure you have the correct path to your driver)
def init_selenium():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")  # Run headless for efficiency
    service = Service(r'chromedriver.exe')  # Update this with your driver path
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# Initialize selenium and create driver
driver = init_selenium()

# Step 1: Authenticate and access Google Sheets
def access_google_sheet(sheet_url, sheet_path):
    creds = Credentials.from_service_account_file(sheet_path, scopes=SCOPES)
    client = gspread.authorize(creds)
    sheet = client.open_by_url(sheet_url).sheet1  # assuming single sheet
    return sheet

# Step 2: Check if the website is built with React (or any dynamic JS frameworks)
def is_dynamic_website(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Check for signs of React, Vue, Angular like minimal <body> or placeholder elements
    body_content = soup.body.get_text(strip=True)
    
    if not body_content or body_content.lower() in ['loading', '', ' ']:  # Minimal body content indicates JS-driven
        return True

    # Look for React/Vue.js/Angular signatures (e.g., <div id="root"> or data-react* attributes)
    if soup.find(attrs={"id": "root"}) or soup.find(attrs={"id": "app"}) or soup.find(attrs={"data-reactroot": True}):
        return True
    
    return False

# Step 3: Scroll and wait for lazy loading if necessary (React-based sites)
def fetch_full_page_html(url):
    driver.get(url)
    
    # Scroll down gradually to simulate user interaction and ensure all lazy-loaded content appears
    scroll_pause_time = 2
    screen_height = driver.execute_script("return window.screen.height;")
    for i in range(3):  # Scroll multiple times (adjust as necessary)
        driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        time.sleep(scroll_pause_time)
    
    # After scrolling, return the fully loaded HTML
    return driver.page_source

# Step 4: Extract relevant HTML sections (header, footer, contact sections)
def extract_relevant_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract <header>, <footer>, and likely contact sections
    relevant_sections = []

    header = soup.find('header')
    if header:
        relevant_sections.append(header.get_text(separator=" ", strip=True))

    footer = soup.find('footer')
    if footer:
        relevant_sections.append(footer.get_text(separator=" ", strip=True))

    contact_section = soup.find(attrs={"id": re.compile(r"contact|about", re.I)})
    if contact_section:
        relevant_sections.append(contact_section.get_text(separator=" ", strip=True))
    
    # Add any divs or sections that look like contact information
    for section in soup.find_all('div', class_=re.compile(r"contact|about|footer", re.I)):
        relevant_sections.append(section.get_text(separator=" ", strip=True))

    # Join all sections into a single text string and limit the size
    relevant_text = " ".join(relevant_sections)
    
    # Ensure the length is within a reasonable token limit (assume ~4 characters per token)
    max_chars = 2000  # Adjust as needed to stay under 500 tokens
    return relevant_text[:max_chars]

# Step 5: Scrape data from a domain and pass HTML to LLM for extraction
def scrape_website(domain):
    data = {
        "Domain Name": domain,
        "Owner's Name": "",
        "Contact Email": "",
        "Phone Number": "",
        "Facebook": "",
        "Pinterest": "",
        "Twitter": "",
        "Youtube": "",
        "Instagram": "",
        "Linkedin": ""
    }
    
    try:
        url = f"http://{domain}"
        
        # If the website is React-based or uses JavaScript for dynamic content, use Selenium
        response = requests.get(url, timeout=10)
        if is_dynamic_website(response.text):
            print(f"Dynamic website detected for {domain}, using Selenium to fetch content.")
            full_html = fetch_full_page_html(url)
        else:
            full_html = response.text

        # Extract relevant sections of HTML before passing to LLM
        relevant_html = extract_relevant_html(full_html)

        # Pass relevant HTML content to the LLM for extraction
        llm_response = check_with_llm(relevant_html)
        data["Owner's Name"] = llm_response.get("Owner's Name", "")
        data["Contact Email"] = llm_response.get("Contact Email", "")
        data["Phone Number"] = llm_response.get("Phone Number", "")

        # Now scrape the social media links
        soup = BeautifulSoup(full_html, 'html.parser')
        for link in soup.find_all('a', href=True):
            url = link['href']
            if 'facebook.com' in url:
                data['Facebook'] = url
            elif 'pinterest.com' in url:
                data['Pinterest'] = url
            elif 'twitter.com' in url:
                data['Twitter'] = url
            elif 'youtube.com' in url:
                data['Youtube'] = url
            elif 'instagram.com' in url:
                data['Instagram'] = url
            elif 'linkedin.com' in url:
                data['Linkedin'] = url

    except requests.exceptions.RequestException as e:
        print(f"Error scraping {domain}: {e}")
        return None
    
    return data

def update_google_sheet(sheet, row, data):
    # Update each cell for the current row
    sheet.update(f"B{row}", [[data['Domain Name']]])
    sheet.update(f"C{row}", [[data["Owner's Name"]]])
    sheet.update(f"D{row}", [[data['Contact Email']]])
    sheet.update(f"E{row}", [[data['Phone Number']]])
    sheet.update(f"F{row}", [[data['Facebook']]])
    sheet.update(f"G{row}", [[data['Pinterest']]])
    sheet.update(f"H{row}", [[data['Twitter']]])
    sheet.update(f"I{row}", [[data['Youtube']]])
    sheet.update(f"J{row}", [[data['Instagram']]])
    sheet.update(f"K{row}", [[data['Linkedin']]])

# Main Function with throttling and row-by-row updating
def main(sheet_url, sheet_path, start_row=2):
    sheet = access_google_sheet(sheet_url, sheet_path)
    rows = sheet.get_all_records()

    api_calls = 0
    api_call_limit = 55  # Limit to 55 calls per minute
    start_time = time.time()  # Record the starting time for rate limiting

    # Adjust the indexing to start from the specified row
    for idx, row in enumerate(rows[start_row - 2:], start=start_row):  # Adjust index based on start_row
        domain = row['Domain']
        print(f"Scraping domain: {domain}")

        scraped_data = scrape_website(domain)

        # If scraping is successful, update the sheet row by row
        if scraped_data:
            update_google_sheet(sheet, idx, scraped_data)
            
            # Count the API calls
            api_calls += 11  # 10 data columns (B-K) + 1 HTML column (L)

            # Throttle if we are reaching the API limit
            elapsed_time = time.time() - start_time
            if api_calls >= api_call_limit:
                if elapsed_time < 60:
                    sleep_time = 60 - elapsed_time
                    print(f"API limit reached. Pausing for {sleep_time} seconds...")
                    time.sleep(sleep_time)  # Wait for the remaining time in the minute
                # Reset after 60 seconds
                start_time = time.time()
                api_calls = 0

In [12]:
# Script Execution
if __name__ == "__main__":
    main(sheet_url, sheet_path, start_row=130)

Scraping domain: woodworkingshop.com


C:\Users\hp\AppData\Local\Temp\ipykernel_18104\3400015349.py:139: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(f"B{row}", [[data['Domain Name']]])
C:\Users\hp\AppData\Local\Temp\ipykernel_18104\3400015349.py:140: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(f"C{row}", [[data["Owner's Name"]]])
C:\Users\hp\AppData\Local\Temp\ipykernel_18104\3400015349.py:141: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(f"D{row}", [[data['Contact Email']]])
C:\Users\hp\AppData\Local\Temp\ipykernel_18104\3400015349.py:142: DeprecationWarning: The order of arguments in worksheet.update() ha

Scraping domain: woodworkingcorner.com
Scraping domain: woodworkforums.com
Scraping domain: woodworkersworkshop.com
Scraping domain: woodworkersjournal.com
API limit reached. Pausing for 16.752262115478516 seconds...
Scraping domain: woodworkersinstitute.com
Scraping domain: wooduchoose.com
Scraping domain: woodturnerscatalog.com
Scraping domain: woodtalkonline.com
Scraping domain: woodstore.net
API limit reached. Pausing for 24.453282117843628 seconds...
Scraping domain: woodstockvintagelumber.com
Scraping domain: woodstockoutlet.com
Scraping domain: woodstockchimes.myshopify.com
Scraping domain: woodsmithplans.com
Scraping domain: woodsmith.com
API limit reached. Pausing for 25.10023808479309 seconds...
Scraping domain: woodshopdiaries.com
Scraping domain: woodshop-diaries.myshopify.com
Scraping domain: woodridgelibrary.org
Scraping domain: woodpeckerscrafts.com
Scraping domain: woodnet.net
API limit reached. Pausing for 26.811496019363403 seconds...
Scraping domain: woodmagazine.com